In [1]:
!git clone https://github.com/rahulvks/ponniyin-selvan-

Cloning into 'ponniyin-selvan-'...
remote: Enumerating objects: 341, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 341 (delta 0), reused 0 (delta 0), pack-reused 338
Receiving objects: 100% (341/341), 1.79 MiB | 11.23 MiB/s, done.
Resolving deltas: 100% (18/18), done.


In [35]:
!pip install indic-nlp-library

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 11.3 MB/s eta 0:00:00


In [14]:
import torch
import torch.nn as nn
try:
  import einops
  from einops import rearrange,reduce,repeat
except:
  !pip install einops
  import einops
  from einops import rearrange,reduce,repeat
import torch.nn.functional as F
import math

In [61]:
import os
import glob
import numpy as np
import pickle
from tqdm.notebook import tqdm
def sorted_value(x):
  return float(x.split('/')[-2].split('-')[-1]),int(x.split('/')[-1].split('_')[-1].split('.')[0])
text_files = sorted(glob.glob('/content/ponniyin-selvan-/**/*.txt'),key=sorted_value)
total_corpus=''
numpy_tokens=[]
for file in text_files:
  with open(file,'r') as f:
    words=f.readline()
    words=words.split(' ')[3:]
    current_word= ' '.join(words)
    arr=tokenizer(current_word,add_special_tokens=False).input_ids
    numpy_tokens=arr if numpy_tokens==[] else numpy_tokens+arr


"""with open("total_corpus.txt",'w') as f:
  f.write(total_corpus)"""

'with open("total_corpus.txt",\'w\') as f:\n  f.write(total_corpus)'

In [77]:
numpy_tokens=np.array(numpy_tokens,dtype=np.int16)
with open('tokens.npy','wb') as f:
  np.save(f,numpy_tokens)

In [8]:
block_size=32

In [31]:
class MHA(nn.Module):
  def __init__(self,n_heads,n_embd,attention_drop=0):
    super().__init__()
    self.head_size=n_embd//n_heads
    self.n_heads=n_heads
    self.attn_drop=attention_drop
    self.qkv=nn.Linear(n_embd,n_embd*3)
    self.dropout=nn.Dropout(self.attn_drop)
    self.out=nn.Linear(n_embd,n_embd)

  def forward(self,x):
    B,T,C=x.shape
    C//=self.n_heads
    qkv= rearrange(self.qkv(x),'B T (C s)-> s B T C',s=self.n_heads)
    q,k,v=rearrange(qkv,'heads B T (C split)-> split B heads T C',split=3)
    out=F.scaled_dot_product_attention(q,k,v,is_causal=True)
    out=rearrange(out,"B heads T C -> B T (C heads)")
    return self.out(out)


class FeedForward(nn.Module):

  def __init__(self,n_embd,dropout=0):
    super().__init__()
    self.net=nn.Sequential(
        nn.Linear(n_embd,4*n_embd),
        nn.GELU(),
        nn.Linear(4*n_embd,n_embd),
        nn.Dropout(dropout)
    )

  def forward(self,x):
    return self.net(x)


class Block(nn.Module):
  def __init__(self,n_embd,n_head,dropout=0):
    super().__init__()

    self.sa = MHA(n_head,n_embd,dropout)
    self.ffwd= FeedForward(n_embd,dropout)

    self.ln1= nn.LayerNorm(n_embd)
    self.ln2=nn.LayerNorm(n_embd)


  def forward(self,x):
    sa=self.sa(self.ln1(x))

    x= x+ sa
    x= x+ self.ffwd(self.ln2(x))
    return x

In [32]:
model=Block(768,8)

In [40]:
from transformers import AlbertTokenizer

tokenizer = AlbertTokenizer.from_pretrained("ai4bharat/IndicBART", do_lower_case=False, use_fast=False, keep_accents=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


pytorch_model.bin:   0%|          | 0.00/976M [00:00<?, ?B/s]

In [69]:
input_sentence = "ந"
inp = tokenizer(input_sentence, add_special_tokens=False, return_tensors="pt", padding=True).input_ids
print("Original input sentence:", input_sentence)
print("Segmented input sentence:", tokenizer.convert_ids_to_tokens(inp[0]))
print("Orginal Again:",tokenizer.decode(inp[0]))
print("Input sentence as tensor: ", inp)

Original input sentence: ந
Segmented input sentence: ['▁', 'ந']
Orginal Again: ந
Input sentence as tensor:  tensor([[   41, 60827]])


In [ ]:
"""total_token_count in INDICBART is  64014  so we will Intialize the particular Embedding matrix with the same number of tokens and with decoder only block and finally with Linear layer,
Pretraining from the Poniyan selvan tamil corpus"""

In [ ]:
def position_sin_cos(n_emb,context):

  assert n_emb%2

  pe=torch.zeros(context,n_emb)
  position=torch.arange(0,length).unsqueeze(1).float()
  div_term=torch.exp((torch.arange(0,n_emb,2,dtype=torch.float)) * -(math.log(10000)/d_model))

  pe[:,0::2] =  torch.sin(position*div_term)
  pe[:,1::2] =  torch.cos(position*div_term)

  return pe


In [73]:
class MultiToken_prediction(nn.Module):
  def __init__(self,n_emb,n_heads,drop_out_blocks=0.0,n_heads_last=1,layers=4):

    self.vocab_size=64014
    self.n_emb=n_emb
    self.context_size=context_size # for now we make it sin cos
    self.n_layers=layers
    self.n_heads_last=n_heads_last

    self.pe=position_sin_cos(self.n_emb,self.n_context)

    self.vocab_size=nn.Embedding(self.vocab_size,self.n_emb)

    self.shared_trunk=nn.ModuleList(
        [Block(n_emb,n_heads) for _ in range(layers)]
    )

    self.independent_heads(
        [MHA(n_head,n_emb) for i in range(layers)]
    )
    self.ln_norm=nn.LayerNorm(self.n_emb)
    self.lm_head=nn.Linear(self.n_emb,self.vocab_size,bias=False)

    self.vocab_size.weights=self.lm_head.weights

    self.apply(_init_weights)

  def _init_weights(self,module):
    if isinstance(module,nn.Linear):
      std=0.02 * (2*self.layers) ** -0.5

      torch.nn.init.normal_(module.weight,mean=0.0,std=std)

      if module.bias is not None:
        torch.nn.init.zeros_(module.bias)
    elif isinstance(module,nn.Embedding):
      torch.nn.init.normal_(module.weight,mean=0.0,std=0.02)

  def forward_shared_trunk(self,idx):

    batch,context=idx.shape

    x= self.vocab_size(idx) + self.pe

    for block in self.shared_trunk:
      x=block(x)

    x=self.ln_norm(x)

    return x

SyntaxError: incomplete input (<ipython-input-73-77b81fa9d043>, line 37)

In [ ]:
class Dataloader:
  def __init__(self,B,T,tokens_to_pred):
    self.B=B
    self.T=T
    self.current_position=0
    self.tokens_to_pred=tokens_to_pred
    self.reset()
  def reset(self):
    self.current_position=0
    np_tokens=np.load("tokens.npy")
    np_tokens=np_tokesn.astype(np.int32)
    ptt=torch.tensor(np_tokens,dtype=torch.long)

  def next_batch(self):
    B,T=self.B,self.T
    buf=self.tokens[self.current_position:self.current_position+B*T+4]
    X=buf[:-self.tokens_to_pred].view(B,T)
    Y=buf[1:]
    self.current_position+=B*T

    if self.current_position + (B*T+self.tokens_to_pred) > len(self.tokens):
      self.reset()

    return X,Y




In [79]:
total_batch_size=2**17
B=16
T=1024
grad_accumulate_steps= total_batch_size // (B*T)




131072

In [81]:
131072//(64*1024)

2

In [ ]:
def train_loop(model,optimizer,total_grad_accumulation,train_loader,device):
  for microstep in range(grad_accumulate_steps):
    x,y=train_loader.next_batch()
    x,y=x.to(device),y.to(device)

    with torch.autocast(device_type=device,dtype=torch.bfloat16):
      logits=model.forward()